# 一、读取处理数据

In [176]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

volume = pd.read_excel('TRD_Dalyr.xlsx').loc[2:,:]
volume1 = pd.read_excel('TRD_Dalyr1.xlsx').loc[2:,:]
ret = pd.read_excel('TRD_Mnth.xlsx').loc[2:,:]
ret['Newmnt'] = ret['Trdmnt'].apply(lambda x: int(x.replace('-','')))
ret['Mretwd'] = ret['Mretwd'].astype('float64')
volume = pd.concat([volume, volume1], axis=0).reset_index(drop=True) #合并两个成交量数据
volume['Newmnt'] = volume['Trddt'].apply(lambda x: int(int(x[0:4]+x[5:7])))

C:\ProgramData\Anaconda3\Lib\site-packages\openpyxl\styles\stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")
C:\ProgramData\Anaconda3\Lib\site-packages\openpyxl\styles\stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")
C:\ProgramData\Anaconda3\Lib\site-packages\openpyxl\styles\stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


# 二、计算volume_factor因子

In [177]:
#reference period：2个月；formation period：第二个月的最后一天；testing period：第三个月的收益率
#从2018年11月开始不断rolling
dates = volume['Newmnt'].drop_duplicates().tolist()
allLL_factor = pd.DataFrame()
for i in range(2,len(dates)):
    #筛选出符合条件的股票组合
    print(dates[i])
    temp1 = volume[volume['Newmnt'].isin(dates[i-2:i+1])].groupby('Stkcd').count()
    stocks_list = temp1[temp1['Newmnt']==temp1['Newmnt'].max()].index.tolist()
    volume_factor= {}
    
    volume_temp = volume[(volume['Stkcd'].isin(stocks_list))&(volume['Newmnt']).isin(dates[i-2:i])].copy() 
    volume_temp.loc[:,'volume_factor'] = volume_temp.groupby('Stkcd')['Dnvaltrd'].transform(lambda x: x.rank()[-1:].values[0]/len(x)) #最后一天的交易量在前两个月的分位值
    volume_factor = volume_temp.loc[:,['Stkcd','volume_factor']].drop_duplicates().reset_index(drop=True)
    volume_factor['Newmnt'] = dates[i]
    
    allLL_factor=pd.concat((allLL_factor,volume_factor),axis=0)

## 当前期的因子和下一期的return做合并
testdata=pd.merge(ret,allLL_factor.reset_index(),how='inner',on=['Stkcd','Newmnt'])
testdata

201901
201902
201903
201904
201905
201906
201907
201908
201909
201910
201911
201912
202001
202002
202003
202004
202005
202006
202007
202008
202009
202010
202011
202012
202101
202102
202103
202104
202105
202106
202107
202108
202109
202110
202111
202112
202201
202202
202203
202204
202205
202206
202207
202208
202209
202210
202211
202212
202301
202302
202303
202304
202305
202306
202307
202308
202309
202310


,Stkcd,Trdmnt,Mretwd,Newmnt,index,volume_factor
0,600000,2019-01,0.094898,201901,0,0.809524
1,600000,2019-02,0.094129,201902,0,0.833333
2,600000,2019-03,-0.039182,201903,0,0.918919
3,600000,2019-04,0.061170,201904,0,0.861111
4,600000,2019-05,-0.070175,201905,0,1.000000
...,...,...,...,...,...,...
88483,605599,2023-06,0.186193,202306,1623,0.205128
88484,605599,2023-07,0.053747,202307,1642,0.700000
88485,605599,2023-08,-0.025862,202308,1656,0.585366
88486,605599,2023-09,0.072271,202309,1663,0.431818


# 三、分位数组合volume因子有效性检验

In [178]:
import statsmodels.api as sm

def getgroup(x):
    return pd.qcut(x,10,labels=False,duplicates='drop')

#decile portfolios
testdata['group'] = testdata.groupby('Newmnt')['volume_factor'].rank(method='first',pct=True).apply(lambda x : int(10*x))
gret = testdata.groupby(['Newmnt','group'])['Mretwd'].mean()
gretnew = gret.reset_index().pivot_table(index='Newmnt',columns='group',values='Mretwd')
gretnew.loc[:,'spread']=gretnew.loc[:,9]-gretnew.loc[:,0]

from scipy import stats
print('单因素t检验：')
print(stats.ttest_1samp(gretnew['spread'], 0))

tmpdata=sm.add_constant(gretnew['spread'])
tmpdata

mod = sm.OLS(tmpdata.iloc[:,1], tmpdata.iloc[:,0])
reshac=mod.fit(cov_type='hac',cov_kwds={'maxlags':2}) #Newey-west estimator
print(reshac.summary())

单因素t检验：
Ttest_1sampResult(statistic=-2.0428924512611406, pvalue=0.04569696285977001)
                            OLS Regression Results                            
Dep. Variable:                 spread   R-squared:                       0.000
Model:                            OLS   Adj. R-squared:                  0.000
Method:                 Least Squares   F-statistic:                       nan
Date:                Wed, 01 Nov 2023   Prob (F-statistic):                nan
Time:                        15:56:38   Log-Likelihood:                 117.76
No. Observations:                  58   AIC:                            -233.5
Df Residuals:                      57   BIC:                            -231.5
Df Model:                           0                                         
Covariance Type:                  hac                                         
                 coef    std err          z      P>|z|      [0.025      0.975]
----------------------------------------------

# 四、收益对因子回归

In [179]:
def dofmreg(data):
    mod = sm.OLS(data.loc[:,'Mretwd'], sm.add_constant(data.loc[:,'volume_factor']))
    res = mod.fit()
    return res.params

testdata['Mretwd'] = testdata['Mretwd'].astype('float64')
regres=testdata.groupby('Newmnt').apply(dofmreg)
print('单因素t检验：')
print(stats.ttest_1samp(regres['volume_factor'], 0))

单因素t检验：
Ttest_1sampResult(statistic=-2.4828771117146307, pvalue=0.01599997149512734)
